![New Release: Accelerate YOLOv8](assets/yolov8.png)

# Accelerate Ultralytics YOLOv8 with Speedster

In [17]:
# model_name: str = "yolov8-speedster-seg.pt"
model_name: str = './yolov8-speedster.pt'

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the Speedster module from the open-source library nebullvm.

With Speedster's latest API, you can speed up models up to 10 times without any loss of accuracy (option A), or accelerate them up to 20-30 times by setting a self-defined amount of accuracy/precision that you are willing to trade off to get even lower response time (option B). To accelerate your model, Speedster takes advantage of various optimization techniques such as deep learning compilers (in both option A and option B), quantization, half accuracy, and so on (option B).

Let's jump to the code.

In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## Setup

### Install Speedster

In [3]:
!pip install speedster

In [4]:
!python -m nebullvm.installers.auto_installer --frameworks torch --compilers all

2023-03-21 07:39:49 | INFO     | Running auto install of nebullvm dependencies
--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segme

### Install Ultralytics YOLOv8

In [5]:
!pip install ultralytics


## Load YOLOv8s

In [6]:
import torch
from ultralytics import YOLO

yolo = YOLO(model_name)

/Users/earyzhe/opt/anaconda3/envs/NERF_TURRET/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's load a test dummy data and see the original output

In [7]:
test_data = torch.randn(1, 3, 640, 640)
yolo.model(test_data) # type: ignore

(tensor([[[4.96410e+00, 2.36664e+01, 3.18440e+01,  ..., 5.30065e+02, 5.44457e+02, 5.65200e+02],
          [1.09848e+01, 1.17072e+01, 8.16328e+00,  ..., 6.15886e+02, 6.09443e+02, 6.09050e+02],
          [9.94893e+00, 4.57384e+01, 5.78672e+01,  ..., 2.14935e+02, 2.09363e+02, 1.97493e+02],
          ...,
          [7.92733e-07, 5.45574e-07, 4.10054e-07,  ..., 2.26634e-06, 1.30161e-06, 8.79081e-07],
          [4.39973e-07, 2.82247e-07, 2.41645e-07,  ..., 2.18358e-06, 1.82922e-06, 1.41129e-06],
          [5.40796e-07, 1.86909e-07, 1.62783e-07,  ..., 1.92714e-06, 1.48381e-06, 1.14982e-06]]]),
 [tensor([[[[  8.66436,   4.65574,   3.34604,  ...,   1.19562,   4.71748,   6.55182],
            [  9.32375,   5.00959,   2.85007,  ...,   2.20513,   6.35949,   7.74808],
            [  9.22110,   4.85481,   2.60490,  ...,   2.23625,   5.24847,   7.61140],
            ...,
            [  9.32389,   5.21124,   2.56463,  ...,   2.52498,   4.74337,   8.62282],
            [  8.84640,   4.84586,   2.40729,

The original YOLOv8 model return as output a tuple where the first element is a tensor and the second is a list of tensors. Speedster currently supports only models that return only tensors, so we need to create a wrapper to overcome this issue:

In [8]:
class YOLOWrapper(torch.nn.Module):
    def __init__(self, yolo_model):
        super().__init__()
        self.model = yolo_model.model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], *tuple(res[1])
        
model_wrapper = YOLOWrapper(yolo)

## YOLOv8s Optimization with GPU

We can now optimize the model using speedster:

In [9]:
from speedster import optimize_model

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True
)

2023-03-21 07:40:24 | INFO     | Running Speedster on CPU
2023-03-21 07:40:26 | WARNING  | Missing Frameworks: tensorflow.
 Please install them to include them in the optimization pipeline.
2023-03-21 07:40:26 | WARNING  | Missing Compilers: deepsparse, neural_compressor.
 Please install them to include them in the optimization pipeline.
2023-03-21 07:40:26 | WARNING  | Missing Dependencies: onnxmltools.
 Without them, some compilers may not work properly.
2023-03-21 07:41:23 | INFO     | Benchmark performance of original model
2023-03-21 07:42:29 | INFO     | Original model latency: 0.5072965788841247 sec/iter
2023-03-21 07:42:33 | INFO     | Optimizing with PytorchBackendCompiler and q_type: None.
2023-03-21 07:42:33 | WARNING  | Unable to trace model with torch.fx
2023-03-21 07:42:59 | INFO     | Optimized model latency: 0.5444438457489014 sec/iter
2023-03-21 07:42:59 | INFO     | Optimizing with PytorchBackendCompiler and q_type: QuantizationType.DYNAMIC.
2023-03-21 07:42:59 | WARN

We can finally restore the original output format by wrapping the optimized model in a new class:

In [14]:
class OptimizedYOLO(torch.nn.Module):
    def __init__(self, optimized_model):
        super().__init__()
        self.model = optimized_model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], list(res[1:])
    
optimized_wrapper = OptimizedYOLO(optimized_model)

In [11]:
optimized_wrapper(test_data.cuda())

AssertionError: Torch not compiled with CUDA enabled

## YOLOv8s Optimization with CPU

In [15]:
from speedster import optimize_model, save_model, load_model
from ultralytics import YOLO

yolo = YOLO(model_name)
model_wrapper = YOLOWrapper(yolo)

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True, device="cpu"
)

optimized_wrapper = OptimizedYOLO(optimized_model)

FileNotFoundError: [Errno 2] No such file or directory: 'yolov8-speedster.pt'

In [ ]:
optimized_wrapper(test_data)

## Save and reload the optimized model

We can easily save to disk the optimized model with the following line:

In [ ]:
save_model(optimized_model, "model_save_path")

We can then load again the model:

In [16]:
optimized_model = load_model("model_save_path")
optimized_wrapper = OptimizedYOLO(optimized_model)

FileNotFoundError: [Errno 2] No such file or directory: 'model_save_path/metadata.json'

What an amazing result, right?!? Stay tuned for more cool content from the Nebuly team :) 

<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#key-concepts" target="_blank" style="text-decoration: none;"> How speedster works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#quick-start" target="_blank" style="text-decoration: none;"> Quick start </a> 
</center>